### Importación de datos



In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

In [2]:
def cargar_datos(urls:list[str]):
	'''Carga y concatena los datos de varias tiendas en un solo dataframe, agregando una columna 'Tienda_id' para identificar la tienda de origen.'''
	tiendas = []

	for i, url in enumerate(urls):
		tienda = pd.read_csv(url)

		tienda['Tienda_id'] = i + 1

		tiendas.append(tienda)

	return pd.concat(tiendas, ignore_index=True)

In [3]:
urls = [url, url2, url3, url4]

datos = cargar_datos(urls)

In [4]:
datos.tail(2358)

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon,Tienda_id
7077,Lavavajillas,Electrodomésticos,1084800.0,56000.0,11/11/2022,Rafael Acosta,Bogotá,4,Tarjeta de crédito,3,4.60971,-74.08175,4
7078,Cama box,Muebles,473800.0,25400.0,03/01/2020,Izabela de León,Cúcuta,5,Tarjeta de crédito,2,7.89391,-72.50782,4
7079,Balón de baloncesto,Deportes y diversión,23700.0,0.0,25/12/2022,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359,4
7080,Modelado predictivo,Libros,57700.0,1300.0,23/11/2021,Lucas Olivera,Medellín,3,Nequi,1,6.25184,-75.56359,4
7081,Auriculares,Electrónicos,138900.0,5600.0,25/04/2022,Blanca Ramirez,Bogotá,4,Tarjeta de crédito,7,4.60971,-74.08175,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9430,Mesa de comedor,Muebles,227700.0,12600.0,12/02/2020,Camila Rivera,Medellín,5,Tarjeta de crédito,1,6.25184,-75.56359,4
9431,Silla de oficina,Muebles,342000.0,18600.0,17/05/2022,Felipe Santos,Bucaramanga,5,Tarjeta de crédito,3,7.12539,-73.11980,4
9432,Vaso térmico,Deportes y diversión,36700.0,2400.0,03/05/2022,Camila Rivera,Bogotá,5,Tarjeta de crédito,1,4.60971,-74.08175,4
9433,Set de ollas,Artículos para el hogar,151800.0,8300.0,15/06/2020,Blanca Ramirez,Cali,5,Tarjeta de crédito,1,3.43722,-76.52250,4


In [5]:
datos['Tienda_id'].value_counts()

Tienda_id
1    2359
2    2359
3    2359
4    2358
Name: count, dtype: int64

### Verificación de valores nulos
Desciripcion: Buscar en cada una de las tienda la existencia de valores faltantes

In [6]:
# Analizar si hay valores nulos en cada una de las tiendas
valores_nulos = lambda datos: datos.groupby('Tienda_id').apply(lambda x: x.isnull().sum()).transpose()

In [7]:
valores_nulos(datos)

C:\Users\jesmi\AppData\Local\Temp\ipykernel_25920\4224224439.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valores_nulos = lambda datos: datos.groupby('Tienda_id').apply(lambda x: x.isnull().sum()).transpose()


Tienda_id,1,2,3,4
Producto,0,0,0,0
Categoría del Producto,0,0,0,0
Precio,0,0,0,0
Costo de envío,0,0,0,0
Fecha de Compra,0,0,0,0
Vendedor,0,0,0,0
Lugar de Compra,0,0,0,0
Calificación,0,0,0,0
Método de pago,0,0,0,0
Cantidad de cuotas,0,0,0,0


No se encontraron valores faltantes, por lo que no se debe hacer ningún tratamiento 

# 1. Análisis de facturación

In [11]:
''' Función lamda para sumar los datos agrupados por tienda, de la columna especificada y ordenarlos de forma descendente.'''
suma_datos_por_columna = lambda datos, columna: datos.groupby('Tienda_id')[columna].sum().sort_values(ascending=False)

In [12]:
ingreso_total = suma_datos_por_columna(datos, 'Precio')

In [13]:
ingreso_total

Tienda_id
1    1.150880e+09
2    1.116344e+09
3    1.098020e+09
4    1.038376e+09
Name: Precio, dtype: float64

# 2. Ventas por categoría

# 3. Calificación promedio de la tienda


# 4. Productos más y menos vendidos

# 5. Envío promedio por tienda